<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

预备工作：
1. 在text的基础上建dict1  词:频次
2. 遍历(обходим)词典里的每个词，为每一个建list  去掉n个字母的所有形式 （n=1）
3. 每一个得到的形式作为key放到dict2，源词是value；不同value相同的key，value写成list；此处可能有重复key，解决方法是先set()？

算法：
1. 选出词，看它在不在dict1；如果在，直接返回该词
2. 如果不在，建立去掉n个字母的形式的list
3. 在dict2中找每一个上一步得到的形式；如果找到，把value填入list
4. 从得到的list中找到最高频（dict1）；此处在问2中加入3-gram，先3-gram，再高频词
5. 如果词/形式不在任何的字典中，那无法纠正

In [0]:
import os, re # 文件路径库；字符串匹配库
from string import punctuation
import numpy as np # 维度数组和矩阵，针对数组运算的函数
import json # 数据交换格式 "key":value {}object无序 []array有序 value可以是""括起来的string, number, true, false, null, object, array
from collections import Counter
from pprint import pprint # 格式化输出
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)

from sklearn.metrics import classification_report, accuracy_score # 机器学习算法库.矩阵运算；分类，精确度

In [5]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt

--2019-11-22 19:49:56--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123167 (120K) [text/plain]
Saving to: ‘sents_with_mistakes.txt’

sents_with_mistakes 100%[===================>] 120.28K  --.-KB/s    in 0.04s   

2019-11-22 19:49:57 (3.08 MB/s) - ‘sents_with_mistakes.txt’ saved [123167/123167]

--2019-11-22 19:50:08--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awa

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_500.txt

--2019-11-22 19:53:19--  https://github.com/mannefedov/compling_nlp_hse_course/raw/master/data/corpus_500.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt [following]
--2019-11-22 19:53:20--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/corpus_500.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1181403 (1.1M) [text/plain]
Saving to: ‘corpus_500.txt’

corpus_500.txt      100%[===================>]   1.13M  --.-KB/s    in 0.09s   

2019-11-22 19:53:21 (12.9 MB/s) - ‘corpus_500.txt’ saved [1181403/11

In [0]:
def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word in text.lower().split()] # 按空格拆成单词后去标点，此列表项为word
    normalized_text = [word for word in normalized_text if word] # 去除''空项
    return normalized_text

In [0]:
corpus = [] # [["",""],["",""],["",""]] 把词按句子分组
for text in open('corpus_500.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
WORDS = Counter() # corpus词出现的频次
for sent in corpus:
    WORDS.update(sent)
dict_corp = dict(WORDS) # 库词：频次

In [18]:
WORDS.most_common(10)

[('в', 4263),
 ('и', 1999),
 ('на', 1757),
 ('что', 1236),
 ('с', 990),
 ('по', 850),
 ('не', 739),
 ('он', 425),
 ('из', 406),
 ('этом', 405)]

In [0]:
# set去重corpus词
vocab = set() # set会去重，列表update到set：列表中的项添加到set

for sent in corpus:
    vocab.update(sent)


单个库词去掉n个字母的各种形式list

In [0]:
def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву \
    找出差一个字母的选项"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R] # 漏打一个字母
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1 and R[0] != R[1]] # 颠倒两字母顺序
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # 错打成别的字母
    inserts    = [L + c + R               for L, R in splits for c in letters] # 多打一个字母
    return list(set(deletes + transposes + replaces + inserts))

In [0]:
# 建dict_corp2 

In [0]:
def correction(word): 
    "Находим наиболее вероятное похожее слово \
    相似词中最高概率的"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление \
    产生用来对照修正的正确词"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе \
    选出库中有的词"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву \
    找出差一个字母的选项"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R] # 漏打一个字母
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1] # 颠倒两字母顺序
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters] # 错打成别的字母
    inserts    = [L + c + R               for L, R in splits for c in letters] # 多打一个字母
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы \
    找出差两个字母的选项"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1)) # 多循环一遍edits1

In [0]:
for w in dict_corp: 
